In [ ]:
import os
import csv
import psycopg2
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, DATE, TIMESTAMP, VARCHAR, FLOAT, INTEGER
from psycopg2 import sql
import pandas as pd
import create_table_leanx as ctl
import table_schemas

In [ ]:
# ctl.get_raw_query_as_str(table_name='VBAK')

In [ ]:
db_host = 'ce--'
db_port = '--'
db_name = '--'
db_user = '--'
db_password = ''

# Construct the connection string
connection_string = sql.SQL("dbname={} user={} password={} host={} port={}").format(
    sql.Identifier(db_name),
    sql.Identifier(db_user),
    sql.Identifier(db_password),
    sql.Identifier(db_host),
    sql.Identifier(db_port)
)

In [ ]:
connection = psycopg2.connect(f"dbname={db_name} user={db_user} password={db_password} host={db_host} port={db_port}")
cursor = connection.cursor()
# root_folder = "data/OCPM 011124 1858"
root_folder = "data/OCPM/"

ts = table_schemas.table_schemas
ts_csv = table_schemas.columns_to_parse

def find_csv_files(root_folder):
    csv_files = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

def create_or_overwrite_table(csv_file_path, cursor):
    table_name = csv_file_path.split('\\')[-1]
    table_name = table_name.split('.')[0]
    print(f"Creating or overwriting table: {table_name} with data from: {csv_file_path}")

    df = pd.read_csv(csv_file_path, parse_dates=ts_csv[table_name])
    engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    df.to_sql(name=table_name, con=engine, schema='ocpm_om_test', if_exists='replace', index=False, dtype=ts[table_name])

csv_files = find_csv_files(root_folder)
for csv_file in csv_files:
    create_or_overwrite_table(csv_file, cursor)

cursor.close()
connection.close()